In [1]:
import numpy as np
from IPython.display import clear_output
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.datasets import imdb
from keras.utils import np_utils
from keras.models import load_model
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

import import_notebook
from readWrite import ReadWrite
from vectorToDoc import VectorToDoc
from docToVector import DocToVector
from dictionary import Dictionary

Using TensorFlow backend.


importing Jupyter notebook from readWrite.ipynb
importing Jupyter notebook from vectorToDoc.ipynb
importing Jupyter notebook from dictionary.ipynb
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\himur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\himur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
importing Jupyter notebook from docToVector.ipynb


In [2]:
tweetFile = "combineVectors.csv"
tweetClassFile = "combineVectorsResult.txt"

readWrite = ReadWrite()
xdata = readWrite.readFile(tweetFile)
ydata = readWrite.readFileClassifier(tweetClassFile)

x_train = xdata[:int(len(xdata) * 0.8) + 1]
y_train = ydata[:int(len(ydata) * 0.8) + 1]
x_test = xdata[int(len(xdata) * 0.8):-1]
y_test = ydata[int(len(ydata) * 0.8):-1]

print("x_train: {}, shape: {}".format(len(x_train), x_train.shape))
print("y_train: {}, shape: {}".format(len(y_train), y_train.shape))
print("x_test: {}, shape: {}".format(len(x_test), x_test.shape))
print("y_test: {}, shape: {}".format(len(y_test), y_test.shape))

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(ydata)
encoded = encoder.transform(ydata)
# convert integers to dummy variables (i.e. one hot encoded)
encoding = np_utils.to_categorical(encoded)

encoding_train = encoding[:int(len(xdata) * 0.8) + 1]
encoding_test = encoding[int(len(ydata) * 0.8):-1]
print("encoding_train shape: {}".format(encoding_train.shape))
print("encoding_test shape: {}".format(encoding_test.shape))

Read complete, total records: 1814
Read complete, total records: 1814
x_train: 1452, shape: (1452, 20)
y_train: 1452, shape: (1452,)
x_test: 362, shape: (362, 20)
y_test: 362, shape: (362,)
encoding_train shape: (1452, 3)
encoding_test shape: (362, 3)


In [93]:
vocabSize = 2000
outputDim = 64
recordsSize = 30

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

model = Sequential()
model.add(Embedding(vocabSize, outputDim, input_length=recordsSize))
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, input_dim=(20,), activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 64)            128000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 194,303
Trainable params: 194,303
Non-trainable params: 0
_________________________________________________________________


In [94]:
batch_size = outputDim
epochs = 10
scores = 0

# while(True):        
#     model.fit(x_train, encoding_train, validation_data=(x_test, encoding_test), epochs=epochs, batch_size=outputDim, verbose=2)
#     scores = model.evaluate(x_test, encoding_test, verbose=0)    
#     print("Accuracy: %.2f%%" % (scores[1]*100))
#     if(scores[1] > 0.83):
#         break
#     else:
#         clear_output()

model.fit(x_train, encoding_train, validation_data=(x_test, encoding_test), epochs=epochs, batch_size=outputDim, verbose=2)
scores = model.evaluate(x_test, encoding_test, verbose=0)    
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 1452 samples, validate on 362 samples
Epoch 1/10
2s - loss: 0.4707 - acc: 0.7863 - val_loss: 0.4750 - val_acc: 0.7993
Epoch 2/10
1s - loss: 0.3632 - acc: 0.8641 - val_loss: 0.4635 - val_acc: 0.8020
Epoch 3/10
1s - loss: 0.2831 - acc: 0.8882 - val_loss: 0.4301 - val_acc: 0.8204
Epoch 4/10
1s - loss: 0.1982 - acc: 0.9123 - val_loss: 0.4505 - val_acc: 0.8223
Epoch 5/10
2s - loss: 0.1546 - acc: 0.9229 - val_loss: 0.6459 - val_acc: 0.8306
Epoch 6/10
1s - loss: 0.1376 - acc: 0.9261 - val_loss: 0.6409 - val_acc: 0.8260
Epoch 7/10
1s - loss: 0.1245 - acc: 0.9314 - val_loss: 0.8555 - val_acc: 0.8306
Epoch 8/10
1s - loss: 0.1049 - acc: 0.9529 - val_loss: 0.7605 - val_acc: 0.8306
Epoch 9/10
1s - loss: 0.0997 - acc: 0.9591 - val_loss: 0.9763 - val_acc: 0.8352
Epoch 10/10
1s - loss: 0.0844 - acc: 0.9676 - val_loss: 0.7839 - val_acc: 0.8011
Accuracy: 80.11%


In [ ]:
'''
Saving model or load model
'''
# model.save('model/tweetClassifierLaptop.h5')
# model = load_model('model/tweetClassifier1.h5')

In [95]:
'''
Classify sentence
0-unknown, 1-update, 2-delay
'''
dic = Dictionary()
vectorToDoc = VectorToDoc()
docToVector = DocToVector()

sentence = "NSL.. When will this all train signalling fault end"
senArr = dic.readSentence(sentence, True)
print(senArr)

vector = docToVector.convertVector(senArr, True)
sen = vectorToDoc.convertDoc(vector)
vector = np.array(vector)
print(vector)
print(sen)

predicted = model.predict_classes(vector)
print(predicted[0])

Dictionary count: 2377
Dictionary count: 2375
[['nsl', 'train', 'signal', 'fault', 'end']]
Converting records to vectors...
Max length of record: 20
IsPadding enable: True
[[1603 1111 2362 2055 1837    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]
[['nsl', 'train', 'signal', 'fault', 'end', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE']]
1/1 [==============================] - 0s
0


In [96]:
'''
0-unknown, 1-update, 2-delay
'''
testArr = dic.readFile("user_tweets4.txt", True)
vectorsArr2 = docToVector.convertVector(testArr, True)
vectorsArr2 = np.array(vectorsArr2)
oriRecords2 = readWrite.readOriFile('user_tweets4.txt')
# print(vectorsArr2)

# vectorsArr2 = readWrite.readFile('docVectors3.csv')
# oriRecords2 = readWrite.readOriFile('user_tweets3.txt')
print("Shape: {}".format(vectorsArr2.shape))

classResult = []
predicted = model.predict_classes(vectorsArr2)
predicted = np.reshape(predicted, (predicted.size,))

print()
print(predicted.shape)

for idx, score in enumerate(predicted):    
    if(score > 0): 
        classResult.append(oriRecords2[idx])       
        print(predicted[idx])
        print(oriRecords2[idx])        

# readWrite.writeFile('docVectors3Result.txt', predicted)

Converting records to vectors...
Max length of record: 20
IsPadding enable: True
Shape: (62, 20)
32/62 [==============>...............] - ETA: 0s
(62,)
2
@SMRT_Singapore @STcom Wow I've been stuck in between bukit batok and jurong east for 30 minutes already. Thanks for making me miss my movie

2
@SMRT_Singapore @LTAsg How long is your NEW signalling system allowed to malfunction before you declare that it's shit?

2
[NSL]: Due to a train fault on the new signalling system, please add 20mins train travel time between #ChoaChuKang and #JurongEast.

2
@SMRT_Singapore sleep at amk wake up though reach novena BUT still not yet reach toapayoh u are the best

2
@SMRT_Singapore train fault again at #NSL. Stupid service. Train waiting time oled pass 5 minutes lah! please provide correct delay time. Fk!

2
@SMRT_Singapore Haha....kbw open mouth n bless dt3... expect train issue in the coming few days

2
@SMRT_Singapore (Not linked to new signalling project) &lt;-- like that matters.

2
[EWL] UP